# DreamJobber

---

**Process**
1. Clean text
2. Bag of Words
3. LDA model (Latent Dirichlet allocation) 
4. Classification model

---

**Import Necessary Libraries**

In [1]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk

In [2]:
#nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/admin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

---

**Load Data**

In [3]:
df = pd.read_csv('data/combined.csv')

In [4]:
df.head()

,job_description,job_title
0,DOJ offers a range of opportunities for experi...,Attorney and Assistant United States Attorney
1,As an FBI Special Agent with a military or law...,Special Agent - Law Enforcement or Military Ve...
2,As an FBI Special Agent with expertise in educ...,Special Agent - Education/Teaching Background
3,As an FBI Special Agent with Accounting/Financ...,Special Agent - Accounting/Finance Background
4,"As an FBI Special Agent, your STEM background ...","Special Agent - Science, Technology, Engineeri..."


In [5]:
#check for missing values
df.isna().sum()

job_description    5
job_title          0
dtype: int64

In [6]:
#looks like there are 5 rows that have no job description
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000 entries, 0 to 31999
Data columns (total 2 columns):
job_description    31995 non-null object
job_title          32000 non-null object
dtypes: object(2)
memory usage: 500.1+ KB


In [7]:
#drop rows with no job descriptions
df = df[pd.notnull(df['job_description'])]

In [8]:
#sanity check, looks good
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31995 entries, 0 to 31999
Data columns (total 2 columns):
job_description    31995 non-null object
job_title          31995 non-null object
dtypes: object(2)
memory usage: 749.9+ KB


---

## Text Cleaning

1. Tokenize
2. Remove words with fewer than 3 characters
3. Remove stop words
4. Normalize words (Lemmatize and Stem)

In [9]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

**Test the functions on one row of text**

In [10]:
stemmer = SnowballStemmer('english')

In [11]:
text_sample = df[df.index == 5000].values[0][0]

print('original text: ')
words = []
for word in text_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized text: ')
print(preprocess(text_sample))

original text: 
['Newly', 'appointed', 'employee(s)', 'or', 'employee(s)', 'converted', 'to', 'permanent', 'status,', 'selected', 'under', 'this', 'announcement,', 'may', 'be', 'eligible', 'to', 'apply', 'for', 'an', 'award', 'up', 'to', 'the', 'maximum', 'limitation', 'under', 'the', 'provisions', 'of', 'the', 'Education', 'Debt', 'Reduction', 'Program', '(EDRP).', 'Funding,', 'on', 'the', 'final', 'award', 'amount,', 'is', 'contingent', 'on', 'the', 'availability', 'of', 'EDRP', 'funds.', 'Employee(s)', 'must', 'apply', 'for', 'EDRP', 'within', 'four', '(4)', 'months', 'of', 'appointment', 'or', 'conversion.']


 tokenized and lemmatized text: 
['newli', 'appoint', 'employe', 'employe', 'convert', 'perman', 'status', 'select', 'announc', 'elig', 'appli', 'award', 'maximum', 'limit', 'provis', 'educ', 'debt', 'reduct', 'program', 'edrp', 'fund', 'final', 'award', 'conting', 'avail', 'edrp', 'fund', 'employe', 'appli', 'edrp', 'month', 'appoint', 'convers']


**Apply functions to job_description**

In [12]:
#apply function and display first 5 rows
processed_text = df['job_description'].map(preprocess)
processed_text[:5]

0    [offer, rang, opportun, experi, attorney, work...
1    [special, agent, militari, enforc, background,...
2    [special, agent, expertis, educ, gift, relat, ...
3    [special, agent, account, financi, expertis, e...
4    [special, agent, stem, background, provid, ski...
Name: job_description, dtype: object

---

## Bag of Words

In [13]:
#I'll use bag of words to extract features from text for use in modeling

In [31]:
dictionary = gensim.corpora.Dictionary(processed_text)

In [32]:
#check the length before I filter out the extremes
len(dictionary)

56095

In [33]:
dictionary.filter_extremes(no_below=2, no_above=0.5, keep_n=100000)

In [34]:
#check length after filtering out extremes
len(dictionary)

24149

In [35]:
#bow2doc: counts the number of occurrences of each distinct word, 
#converts the word to its integer word id and returns the result as a sparse vector

bow2doc_corpus = [dictionary.doc2bow(text) for text in processed_text]

In [36]:
#let's take a look
bow_doc_5000 = bow2doc_corpus[5000]

for i in range(len(bow_doc_5000)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_5000[i][0], 
                                                     dictionary[bow_doc_5000[i][0]], 
                                                     bow_doc_5000[i][1]))

Word 47 ("militari") appears 2 time.
Word 86 ("area") appears 4 time.
Word 98 ("follow") appears 1 time.
Word 168 ("local") appears 1 time.
Word 193 ("announc") appears 2 time.
Word 196 ("day") appears 1 time.
Word 204 ("open") appears 1 time.
Word 255 ("includ") appears 1 time.
Word 285 ("defin") appears 1 time.
Word 308 ("close") appears 1 time.
Word 315 ("consider") appears 2 time.
Word 316 ("date") appears 1 time.
Word 413 ("vacanc") appears 2 time.
Word 638 ("claim") appears 1 time.
Word 642 ("prefer") appears 1 time.
Word 652 ("member") appears 1 time.
Word 659 ("commut") appears 1 time.
Word 691 ("separ") appears 1 time.
Word 712 ("armi") appears 1 time.
Word 905 ("counti") appears 1 time.
Word 1192 ("move") appears 1 time.
Word 1705 ("citizen") appears 1 time.
Word 2246 ("involuntarili") appears 1 time.
Word 2252 ("spous") appears 1 time.
Word 2290 ("monterey") appears 2 time.
Word 2968 ("garrison") appears 1 time.
Word 3887 ("cruz") appears 1 time.
Word 3888 ("presidio") appea

---

## LDA model with Bag of Words

In [37]:
import time

In [38]:
lda_model = gensim.models.LdaMulticore(bow2doc_corpus, 
                                       num_topics=25, 
                                       id2word=dictionary, 
                                       passes=10, 
                                       workers=4)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


In [39]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.016*"compani" + 0.013*"employ" + 0.013*"oper" + 0.010*"hour" + 0.010*"opportun" + 0.010*"employe" + 0.010*"includ" + 0.009*"shift" + 0.008*"disabl" + 0.008*"applic"
Topic: 1 
Words: 0.025*"manag" + 0.023*"project" + 0.012*"process" + 0.012*"develop" + 0.012*"qualiti" + 0.011*"busi" + 0.011*"skill" + 0.010*"team" + 0.009*"product" + 0.009*"support"
Topic: 2 
Words: 0.013*"offic" + 0.013*"skill" + 0.011*"client" + 0.010*"includ" + 0.009*"assist" + 0.009*"excel" + 0.009*"year" + 0.009*"account" + 0.009*"prepar" + 0.008*"administr"
Topic: 3 
Words: 0.036*"health" + 0.031*"locat" + 0.027*"center" + 0.021*"care" + 0.015*"serv" + 0.014*"provid" + 0.014*"clinic" + 0.014*"area" + 0.014*"veteran" + 0.013*"medic"
Topic: 4 
Words: 0.053*"care" + 0.048*"nurs" + 0.022*"home" + 0.020*"patient" + 0.019*"health" + 0.012*"manag" + 0.012*"regist" + 0.011*"medic" + 0.010*"assist" + 0.010*"provid"
Topic: 5 
Words: 0.032*"secur" + 0.025*"employe" + 0.022*"program" + 0.019*"safeti" + 0.017

In [40]:
#next-steps
#figure out the topics
#use word probability 


---

## LDA model with tf-idf

In [ ]:
#from gensim import corpora, models
#tfidf = models.TfidfModel(bow2doc_corpus)
#corpus_tfidf = tfidf[bow2doc_corpus]
#from pprint import pprint
#for doc in corpus_tfidf:
#    pprint(doc)
#    break

In [ ]:
#lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
 #                                            num_topics=25, 
  #                                           id2word=dictionary, 
   #                                          passes=10, 
    #                                         workers=4)
#for idx, topic in lda_model_tfidf.print_topics(-1):
#    print('Topic: {} Word: {}'.format(idx, topic))